In [12]:
import numpy as np
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.ndimage import rotate, affine_transform
import cv2

def abrir_imagem(caminho):
    img = Image.open(caminho)
    print(f"Formato: {img.format}, Tamanho: {img.size}, Modo: {img.mode}")
    return img, np.array(img)

def plotar_imagem(img_original, img_filtrada, titulo_original, titulo_filtrada):
    fig, ax = plt.subplots(nrows=1, ncols=2)
    ax[0].imshow(img_original, cmap='gray')
    ax[0].set_title(titulo_original)
    ax[0].axis('off')
    ax[1].imshow(img_filtrada, cmap='gray')
    ax[1].set_title(titulo_filtrada)
    ax[1].axis('off')
    plt.show()

# 3.1 Escala: Redução em 1.5x e aumentar em 2.5x;
def escala_numpy(img, escala): # NumPy
    original_shape = img.shape
    new_shape = (int(original_shape[0] * escala), int(original_shape[1] * escala))
    img_escalada = np.zeros(new_shape, dtype=img.dtype)

    for i in range(new_shape[0]):
        for j in range(new_shape[1]):
            img_escalada[i, j] = img[int(i / escala), int(j / escala)]

    return img_escalada

def escala_pillow(img, escala): # Pillow
    new_size = (int(img.width * escala), int(img.height * escala))
    return np.array(img.resize(new_size))

def escala_opencv(img, escala): # OpenCv
    return cv2.resize(img, None, fx=escala, fy=escala)

def escala_scipy(img, escala): # SciPy
    from skimage.transform import rescale
    return (rescale(img, escala) * 255).astype(np.uint8)


# 3.2 Rotação em 45º, 90º e 100º;
def rotacao_numpy(img, angulo): # NumPy
    angle_rad = np.deg2rad(angulo)
    a, l = img.shape
    new_a = int(abs(a * np.cos(angle_rad)) + abs(l * np.sin(angle_rad)))
    new_l = int(abs(a * np.sin(angle_rad)) + abs(l * np.cos(angle_rad)))

    img_rotacionada = np.zeros((new_a, new_l), dtype=img.dtype)
    center_x, center_y = l // 2, a // 2
    new_center_x, new_center_y = new_l // 2, new_a // 2

    for i in range(a):
        for j in range(l):
            new_i = int((i - center_y) * np.cos(angle_rad) - (j - center_x) * np.sin(angle_rad) + new_center_y)
            new_j = int((i - center_y) * np.sin(angle_rad) + (j - center_x) * np.cos(angle_rad) + new_center_x)
            if 0 <= new_i < new_a and 0 <= new_j < new_l:
                img_rotacionada[new_i, new_j] = img[i, j]

    return img_rotacionada

def rotacao_pillow(img, angulo): # Pillow
    return np.array(img.rotate(angulo))

def rotacao_opencv(img, angulo): # OpenCv
    a, l = img.shape[:2]
    center = (l // 2, a // 2)
    matrix = cv2.getRotationMatrix2D(center, angulo, 1.0)

    return cv2.warpAffine(img, matrix, (l, a))

def rotacao_scipy(img, angulo): # SciPy
    return np.clip(rotate(img, angulo, reshape=False), 0, 255).astype(int)

# 3.4 Translação em 35 pixel no eixo X, 45 eixo Y;
def translacao_numpy(img, x, y): # NumPy
    a, l = img.shape
    img_translada = np.zeros_like(img)

    for i in range(a):
        for j in range(l):
            new_i = i + y
            new_j = j + x
            if 0 <= new_i < a and 0 <= new_j < l:
                img_translada[new_i, new_j] = img[i, j]

    return img_translada

def translacao_pillow(img, x, y): # Pillow
    img_pil = Image.fromarray(np.array(img), mode='L')
    return np.array(img_pil.transform(img_pil.size, Image.AFFINE, (1, 0, x, 0, 1, y), resample=Image.NEAREST))

def translacao_opencv(img, x, y): # OpenCv
    a, l = img.shape[:2]
    M = np.float32([[1, 0, x], [0, 1, y]])
    return cv2.warpAffine(img, M, (l, a))

def translacao_scipy(img, x, y): # SciPy
    transform = np.array([[1, 0, x], [0, 1, y]])
    return np.clip(affine_transform(img, transform[:2, :2], offset=transform[:2, 2]), 0, 255).astype(int)

def main():
    imagens = ['lena_gray_512.tif', 'cameraman.tif', 'house.tif']

    for caminho in imagens:
        img_original, img_cv = abrir_imagem(caminho)

        # Escala
        img_escalada_reducao_numpy = escala_numpy(img_cv, 1 / 1.5)
        img_escalada_aumento_numpy = escala_numpy(img_cv, 2.5)
        img_escalada_reducao_pillow = escala_pillow(img_original, 1 / 1.5)
        img_escalada_aumento_pillow = escala_pillow(img_original, 2.5)
        img_escalada_reducao_opencv = escala_opencv(img_cv, 1 / 1.5)
        img_escalada_aumento_opencv = escala_opencv(img_cv, 2.5)
        img_escalada_reducao_scipy = escala_scipy(img_cv, 1 / 1.5)
        img_escalada_aumento_scipy = escala_scipy(img_cv, 2.5)

        plotar_imagem(img_original, img_escalada_reducao_numpy, "Original", "Escalada Redução 1.5x (Numpy)")
        plotar_imagem(img_original, img_escalada_aumento_numpy, "Original", "Escalada Aumento 2.5x (Numpy)")
        plotar_imagem(img_original, img_escalada_reducao_pillow, "Original", "Escalada Redução 1.5x (Pillow)")
        plotar_imagem(img_original, img_escalada_aumento_pillow, "Original", "Escalada Aumento 2.5x (Pillow)")
        plotar_imagem(img_original, img_escalada_reducao_opencv, "Original", "Escalada Redução 1.5x (OpenCV)")
        plotar_imagem(img_original, img_escalada_aumento_opencv, "Original", "Escalada Aumento 2.5x (OpenCV)")
        plotar_imagem(img_original, img_escalada_reducao_scipy, "Original", "Escalada Redução 1.5x (SciPy)")
        plotar_imagem(img_original, img_escalada_aumento_scipy, "Original", "Escalada Aumento 2.5x (SciPy)")

        # Rotação
        img_rotacionada_45_numpy = rotacao_numpy(img_cv, 45)
        img_rotacionada_90_numpy = rotacao_numpy(img_cv, 90)
        img_rotacionada_100_numpy = rotacao_numpy(img_cv, 100)
        img_rotacionada_45_pillow = rotacao_pillow(img_original, 45)
        img_rotacionada_90_pillow = rotacao_pillow(img_original, 90)
        img_rotacionada_100_pillow = rotacao_pillow(img_original, 100)
        img_rotacionada_45_opencv = rotacao_opencv(img_cv, 45)
        img_rotacionada_90_opencv = rotacao_opencv(img_cv, 90)
        img_rotacionada_100_opencv = rotacao_opencv(img_cv, 100)
        img_rotacionada_45_scipy = rotacao_scipy(img_cv, 45)
        img_rotacionada_90_scipy = rotacao_scipy(img_cv, 90)
        img_rotacionada_100_scipy = rotacao_scipy(img_cv, 100)

        plotar_imagem(img_original, img_rotacionada_45_numpy, "Original", "Rotacionada 45º (Numpy)")
        plotar_imagem(img_original, img_rotacionada_90_numpy, "Original", "Rotacionada 90º (Numpy)")
        plotar_imagem(img_original, img_rotacionada_100_numpy, "Original", "Rotacionada 100º (Numpy)")
        plotar_imagem(img_original, img_rotacionada_45_pillow, "Original", "Rotacionada 45º (Pillow)")
        plotar_imagem(img_original, img_rotacionada_90_pillow, "Original", "Rotacionada 90º (Pillow)")
        plotar_imagem(img_original, img_rotacionada_100_pillow, "Original", "Rotacionada 100º (Pillow)")
        plotar_imagem(img_original, img_rotacionada_45_opencv, "Original", "Rotacionada 45º (OpenCV)")
        plotar_imagem(img_original, img_rotacionada_90_opencv, "Original", "Rotacionada 90º (OpenCV)")
        plotar_imagem(img_original, img_rotacionada_100_opencv, "Original", "Rotacionada 100º (OpenCV)")
        plotar_imagem(img_original, img_rotacionada_45_scipy, "Original", "Rotacionada 45º (SciPy)")
        plotar_imagem(img_original, img_rotacionada_90_scipy, "Original", "Rotacionada 90º (SciPy)")
        plotar_imagem(img_original, img_rotacionada_100_scipy, "Original", "Rotacionada 100º (SciPy)")

        # Translação em 35 pixel no eixo X, 45 eixo Y;
        img_translacao_numpy = translacao_numpy(img_cv, 35, 45)
        img_translacao_pillow = translacao_pillow(img_original, 35, 45)
        img_translacao_opencv = translacao_opencv(img_cv, 35, 45)
        img_translacao_scipy = translacao_scipy(img_cv, 35, 45)

        # Translação utilizar os parâmetros que quiser nas coordenadas x e y;
        img_translacao_numpy_2 = translacao_numpy(img_cv, 50, 100)
        img_translacao_pillow_2 = translacao_pillow(img_original, 50, 100)
        img_translacao_opencv_2 = translacao_opencv(img_cv, 50, 100)
        img_translacao_scipy_2 = translacao_scipy(img_cv, 50, 100)

        plotar_imagem(img_original, img_translacao_numpy, "Original", "Translada 35px X, 45px Y (Numpy)")
        plotar_imagem(img_original, img_translacao_pillow, "Original", "Translada 35px X, 45px Y (Pillow)")
        plotar_imagem(img_original, img_translacao_opencv, "Original", "Translada 35px X, 45px Y (OpenCV)")
        plotar_imagem(img_original, img_translacao_scipy, "Original", "Translada 35px X, 45px Y (SciPy)")

        plotar_imagem(img_original, img_translacao_numpy_2, "Original", "Translada 50px X, 100px Y (Numpy)")
        plotar_imagem(img_original, img_translacao_pillow_2, "Original", "Translada 50px X, 100px Y (Pillow)")
        plotar_imagem(img_original, img_translacao_opencv_2, "Original", "Translada 50px X, 100px Y (OpenCV)")
        plotar_imagem(img_original, img_translacao_scipy_2, "Original", "Translada 50px X, 100px Y (SciPy)")



if __name__ == "__main__":
    main()

Output hidden; open in https://colab.research.google.com to view.